# Data Augmentation

This notebook describes the process of generating augmented data to mitigate social bias. The steps include loading the original dataset, generating alternative completions (e.g., antistereotypes), and saving the resulting augmented dataset for fine-tuning.


In [1]:
!pip install transformers datasets


In [2]:
import json
from pathlib import Path
from datasets import Dataset
from transformers import pipeline
import random


In [3]:
# Upload dans Colab
from google.colab import files
uploaded = files.upload()

# Charger les données
with open("dev.json", "r") as f:
    full_data = json.load(f)

intrasentence_examples = full_data["data"]["intrasentence"]


Saving dev.json to dev.json


In [4]:
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0


In [5]:
examples_augmented = []

for ex in intrasentence_examples:
    stereo = None
    antistereo = None
    for s in ex["sentences"]:
        if s["gold_label"] == "stereotype":
            stereo = s["sentence"]
        elif s["gold_label"] == "anti-stereotype":
            antistereo = s["sentence"]

    if stereo and antistereo:
        # Ajouter les originaux
        examples_augmented.append({"text": stereo, "label": 0})
        examples_augmented.append({"text": antistereo, "label": 1})

        # Générer paraphrases automatiques
        try:
            generated = paraphraser(f"paraphrase: {antistereo}",
                        max_new_tokens=32,  # 🔽 beaucoup plus court
                        num_return_sequences=2,
                        do_sample=True)

            for p in generated:
                examples_augmented.append({"text": p['generated_text'], "label": 1})
        except:
            pass  # parfois le modèle plante, on ignore


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
random.shuffle(examples_augmented)
dataset_aug = Dataset.from_list(examples_augmented)
dataset_aug = dataset_aug.train_test_split(test_size=0.1)
dataset_aug.save_to_disk("augmented_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/7581 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/843 [00:00<?, ? examples/s]

In [7]:
from datasets import Dataset
import random

random.shuffle(examples_augmented)
dataset_aug = Dataset.from_list(examples_augmented)
dataset_aug = dataset_aug.train_test_split(test_size=0.1)
dataset_aug.save_to_disk("augmented_dataset")



Saving the dataset (0/1 shards):   0%|          | 0/7581 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/843 [00:00<?, ? examples/s]

In [8]:
!zip -r augmented_dataset.zip augmented_dataset
from google.colab import files
files.download("augmented_dataset.zip")


  adding: augmented_dataset/ (stored 0%)
  adding: augmented_dataset/train/ (stored 0%)
  adding: augmented_dataset/train/state.json (deflated 38%)
  adding: augmented_dataset/train/dataset_info.json (deflated 48%)
  adding: augmented_dataset/train/data-00000-of-00001.arrow (deflated 70%)
  adding: augmented_dataset/test/ (stored 0%)
  adding: augmented_dataset/test/state.json (deflated 38%)
  adding: augmented_dataset/test/dataset_info.json (deflated 48%)
  adding: augmented_dataset/test/data-00000-of-00001.arrow (deflated 67%)
  adding: augmented_dataset/dataset_dict.json (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Summary

We successfully created a dataset with new augmented completions, aiming to improve fairness by reducing stereotypical content. This dataset will be used in the fine-tuning phase of the model training.
